Prompts, chaining, and prompt engineering is important. However, you might not always know what chain or prompts you need to execute prior to receiving user input or new data. This is where automation and **agents** can help. This is an active area of development, but some very useful tooling is available. 

In the following we will explore using [LangChain agents](https://python.langchain.com/en/latest/use_cases/personal_assistants.html) and [Prediction Guard's Task Agents]() to detect and automate LLM actions. 

# Dependencies and imports

In [ ]:
! pip install predictionguard langchain google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=5808a10c238c8def379be294da1863c13130e83056e4f62ac18be75e1a8f90f3
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [21]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import PredictionGuard
import os

# Searching the Internet on-the-fly

In this example, we will use the SerpAPI to perform Google searches on-the-fly based on an LLM parsing of user input. You will need a SerpAPI key to run this example. You can setup a free account and get a key [here](https://serpapi.com/).

In [ ]:
os.environ["SERPAPI_API_KEY"] = "2754b9a56810f58aa9bf68b087c17ca38403974259081f8dea3dd4d5f6c0ff1b"
pgllm = PredictionGuard(token="n4HehSxYpKxQyhKX58IzqPjXa2pOOJ")

In [ ]:
# In LangChain, "tools" are like resources that are available to your agent to
# execute certain actions (like a Google Search) while trying to complete a
# set of tasks. An "agent" is the object that you "run" to try and get a "Final Answer."
tools = load_tools(["serpapi"], llm=pgllm)
agent = initialize_agent(tools, pgllm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("How are Domino's gift cards delivered?")



> Entering new AgentExecutor chain...
 I should look for information about Domino's gift cards.
Action: Search
Action Input: Domino's gift card delivery
Observation: A Dominos Pizza gift card lets you dine in the restaurant or order pizza online to be delivered to your home. Dominos Pizza gift cards can be sent by mail.
Thought: That sounds like the answer.
Final Answer: Domino's gift cards can be sent by mail.

> Finished chain.


"Domino's gift cards can be sent by mail."

In [ ]:
agent.run("Find the Domino's pizza Guarantee. Translate that into German adding a sassy comment about the guarantee.")



> Entering new AgentExecutor chain...
 I need to research Domino's Pizza Guarantee and then translate it
Action: Search
Action Input: Domino's Pizza Guarantee
Observation: Our Guarantee: If you are not completely satisfied with your Domino's Pizza experience, we will make it right or refund your money. Domino's Carryout Insurance ...
Thought: I need to translate this to German
Action: Search
Action Input: Domino's Pizza Guarantee translation German
Observation: Our Guarantee: If you are not completely satisfied with your Domino's Pizza experience, we will make it right or refund your money. Domino's Carryout Insurance ...
Thought: I now know the final answer
Final Answer: Unsere Garantie: Wenn Sie mit Ihrer Domino's Pizza Erfahrung nicht vollständig zufrieden sind, machen wir es richtig oder erstatten Ihnen Ihr Geld zurück. Domino's Carryout Insurance ... Mit der Garantie von Domino's Pizza kannst du sicher sein, dass du immer eine leckere Pizza bekommst - oder dein Geld zurück!

> F

"Unsere Garantie: Wenn Sie mit Ihrer Domino's Pizza Erfahrung nicht vollständig zufrieden sind, machen wir es richtig oder erstatten Ihnen Ihr Geld zurück. Domino's Carryout Insurance ... Mit der Garantie von Domino's Pizza kannst du sicher sein, dass du immer eine leckere Pizza bekommst - oder dein Geld zurück!"

# Decomposing a query automatically into NLP tasks

A complicated user query or data input likely needs to be decomposed into a set of dependent tasks. Prediction Guard's [Task Agent](), which is in beta, can take complicated inputs like the following and decompose them. This notebook just shows the output of this process, and the instructor will run this live.

Rather than giving open domain/ format freedom to an LangChain Agent, the Task Agent from Prediction Guard brings in concrete task structure and type checking to output. This brings a level of consistency and reliability to agents. 

If you are interested in making use of this beta feature in your own work, talk with the instructor, chat with the Prediction Guard team in [Slack](https://join.slack.com/t/prediction-guard/shared_invite/zt-1pwck5pgg-bctLNmqk~CjK3GmJc23UHg), or [book a time for a demo](https://forms.gle/9ZYKkJ9WDLnnvMUw9).

```
INPUT:
-------------------
Tell me if this customer statement toxic: "I hate this product, pure trash you
jackwagons". Respond Yes if toxic and No if not toxic.

TASK DECOMPOSITION:
--------------------
[
  {
    "task": "toxicity detection",
    "id": 0,
    "args": {
      "text": "I hate this product, pure trash you jackwagons"
    }
  },
  {
    "task": "text generation",
    "id": 1,
    "dep": 0,
    "args": {
      "prompt": "Respond Yes if <resource>-0 is toxic and No if not toxic"
    }
  }
]
```

```
INPUT:
-----------------
Find occurances of organization names in this statement: "I love Prediction 
Guard, when making AI predictions I use Prediction Guard. I wouldn't use 
anything but Prediction Guard, so guard yourself, I'm going to promote 
Prediction Guard!". Generate an integer count of the listing of those 
occurances. Generate a number of sassy hot takes equal to that integer count. 
Then translate the hot takes into German

TASK DECOMPOSITION:
----------------------
[
  {
    "task": "named entity recognition",
    "id": 0,
    "args": {
      "phrase": "I love Prediction Guard, when making AI predictions I use Prediction Guard. I wouldn't use anything but Prediction Guard, so guard yourself, I'm going to promote Prediction Guard!"
    }
  },
  {
    "task": "text generation",
    "id": 1,
    "dep": 0,
    "args": {
      "prompt": "Generate an integer count of the listing of those occurances of <resource>-0"
    }
  },
  {
    "task": "text generation",
    "id": 2,
    "dep": 1,
    "args": {
      "prompt": "Generate a number of sassy hot takes equal to <resource>-1"
    }
  },
  {
    "task": "machine translation",
    "id": 3,
    "dep": 2,
    "args": {
      "source": "eng",
      "target": "deu",
      "text": "<resource>-2"
    }
  }
]
```